In [33]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
from sklearn import linear_model

In [34]:
## Data loading procedure
X_2house = pd.read_csv('/wgdisk/ho0338/ek79/Tien/X_2house_v0.csv')
X_2house=X_2house.drop(columns=['Unnamed: 0'])
X_2house.index.rename('Order', inplace=True)

In [35]:
y_2house = -1*np.load('Data/Processed/y_2house.npy')

In [36]:
## check on the data dimensions
print('X_input:',X_2house.shape)
print('y_input:',y_2house.shape)

X_input: (4290985, 158)
y_input: (4290985,)


In [41]:
def select_Neigh_df(df, y, code):
    """This function extracts the samples which belongs to a certian neighborhood"""
    """ Input is dataframe of X_2house comparison, target y array and code of the Neighborhood to be selected"""
    """ return dataframe X_N, corresponding selected y to X_N and the index of the corresponding house chosen(boolean of matching size to y)"""
    code = int(code)
    X_N = df.loc[(df['Neighborhood']==code) & (df['Neighborhood_2']==code)]
    X_N_bol=[(df['Neighborhood']==code) & (df['Neighborhood_2']==code)]  
    y_N = y[X_N_bol]
    return X_N, y_N, X_N_bol

In [42]:
X_15N, y_15N, X_15bol = select_Neigh_df(X_2house, y_2house, 15)

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X_N15,y_N15,test_size=0.3,random_state=22)   ### setup for fitting linear models cv

In [90]:
## training on the ridge models with cv and final fit, with standardization and 
Lasso_m = Lasso()
Ridge_m = Ridge()
LR_m = LinearRegression(fit_intercept = True, normalize= True, X_copy=True)

param_grid_Lasso = {'alpha': np.array([0.01, 0.05 ,0.1, 0.5, 5, 10, 15, 20, 25, 30, 50]), 'fit_intercept':[True], 'normalize':[True]}

In [94]:
def model_fit(X_in, y_in, alpha_ary, alpha_ratio):
    from sklearn.pipeline import Pipeline
    from sklearn.model_selection import train_test_split, KFold
    from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
    from sklearn.preprocessing import StandardScaler
    from sklearn.preprocessing import scale
    from sklearn import metrics
    """ do 4 types of linear model fit with alpha to hypoparam test,  return result matrix"""
    """ Need to input full stratified X data and standardized, matching y """
    fit_intercept = True
    alpha, l1_ratio =.5,.5
    step_Lasso = Pipeline([('scaler',StandardScaler()),('Lasso', Lasso(alpha))]) ## Set up all the pipeline
    step_Ridge = Pipeline([('scaler',StandardScaler()),('Ridge', Ridge(alpha))])
    step_LR= Pipeline([('scaler',StandardScaler()), ('LR', LinearRegression(fit_intercept))])
    step_ELN= Pipeline([('scaler',StandardScaler()), ('ELN', ElasticNet(alpha,l1_ratio))])
        
    # Data prepare:(standardize y only)
    print("mean of prescale y:",np.mean(y_in))
    print("standard deviation of prescale y:", np.std(y_in))
    y_mean = np.mean(y_in)
    y_std = np.std((y_in))
    scal_y_in=scale(y_in)
   
    #set up dataframe for saving results
    
    df=pd.DataFrame(columns=['test#','model type','alpha','alpha_ratio','MSE_test','MSE_train','intercept','cv#'])
    coef_ind = ['test#']+list(X_in.columns)
    coef_df = pd.DataFrame(columns=coef_ind)

    test_no = 1 

    for i in range(len(alpha_ary)):
        alpha= alpha_ary[i]
        
        # divide data to 3 fold and do cross-validation
        kf = KFold(n_splits=3, shuffle=True, random_state=22)
        cv_no=1
        for train_index, test_index in kf.split(X_in):
            X_train, X_test = X_in.iloc[train_index], X_in.iloc[test_index]
            y_train, y_test = scal_y_in[train_index], scal_y_in[test_index]
            # do fitting and predcition
            #(1) Lasso
            #print("cv of Lasso.....", cv_no)
            step_Lasso.fit(X_train,y_train)
            y_test_pred = step_Lasso.predict(X_test)
            y_train_pred = step_Lasso.predict(X_train)
            MSE_test = metrics.mean_squared_error(y_test, y_test_pred)
            MSE_train = metrics.mean_squared_error(y_train, y_train_pred)
            tmp_wrt = [test_no, 'Lasso', alpha, 'NAN', MSE_test, MSE_train, step_Lasso.steps[1][1].intercept_, cv_no]
            df.loc[len(df)]=tmp_wrt
            
            tmp_wrt2 = [test_no]+list(step_Lasso.steps[1][1].coef_)
            coef_df.loc[len(coef_df)]=tmp_wrt2
            
            # re calculate 
            test_no+=1
            
            #(2) Ridge
            print("cv of Ridge.....", cv_no)
            step_Ridge.fit(X_train,y_train)
            y_test_pred = step_Ridge.predict(X_test)
            y_train_pred = step_Ridge.predict(X_train)
            MSE_test = metrics.mean_squared_error(y_test, y_test_pred)
            MSE_train = metrics.mean_squared_error(y_train, y_train_pred)
            tmp_wrt = [test_no, 'Ridge', alpha, 'NAN', MSE_test, MSE_train, step_Ridge.steps[1][1].intercept_, cv_no]
            df.loc[len(df)]=tmp_wrt
            
            tmp_wrt2 = [test_no]+list(step_Ridge.steps[1][1].coef_)
            coef_df.loc[len(coef_df)]=tmp_wrt2
             
            # re calculate             
            test_no+=1
            
            #(3) Linear Regression
           # print("cv of LR.....", cv_no)
            step_LR.fit(X_train,y_train)
            y_test_pred = step_LR.predict(X_test)
            y_train_pred = step_LR.predict(X_train)
            MSE_test = metrics.mean_squared_error(y_test, y_test_pred)
            MSE_train = metrics.mean_squared_error(y_train, y_train_pred)
            tmp_wrt = [test_no, 'LR', 'NAN', 'NAN', MSE_test, MSE_train, step_LR.steps[1][1].intercept_, cv_no]
            df.loc[len(df)]=tmp_wrt
            
            tmp_wrt2 = [test_no]+list(step_LR.steps[1][1].coef_)
            coef_df.loc[len(coef_df)]=tmp_wrt2
             
            # re calculate             
            test_no+=1            

            #(4) Elastic Net
            #print("cv of Elastic Net.....", cv_no)
            for j in range(len(alpha_ratio)):
                l1_ratio = alpha_ratio[j]
                step_ELN.fit(X_train,y_train)
                y_test_pred = step_ELN.predict(X_test)
                y_train_pred = step_ELN.predict(X_train)
                MSE_test = metrics.mean_squared_error(y_test, y_test_pred)
                MSE_train = metrics.mean_squared_error(y_train, y_train_pred)
                tmp_wrt = [test_no, 'ElasticNet', alpha, l1_ratio, MSE_test, MSE_train, step_ELN.steps[1][1].intercept_, cv_no]
                df.loc[len(df)]=tmp_wrt
            
                tmp_wrt2 = [test_no]+list(step_ELN.steps[1][1].coef_)
                coef_df.loc[len(coef_df)]=tmp_wrt2
             
                # re calculate             
                test_no+=1  
            cv_no+=1
            
    p=StandardScaler()
    p.fit(X_in)        
    X_mean = p.mean_
    X_std = np.sqrt(p.var_)

    return df, coef_df, X_mean, X_std, y_mean, y_std

In [48]:
## work on a small validation test to find out what's the relationship btw standardized coeffcients and model inputs
alpha, l1_ratio =.5,.5
Ridge_m0 = Ridge(alpha, fit_intercept=True, normalize=False)  ## no normalization
Ridge_m0.fit(X_train,y_train)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [50]:
Ridge_m0.coef_[1:10]

array([  5.75856511e+03,  -3.18222145e+01,  -1.09148355e+00,
         0.00000000e+00,   2.09575965e+04,   6.20684938e+02,
        -3.54262259e+03,   0.00000000e+00,   3.84034137e+02])

In [51]:
Ridge_m0.intercept_

-1267058.6742524598

In [61]:
## check prediction
pred0=Ridge_m0.predict(X_test[1:10])
print(pred0)

[ 47426.93513101 -40413.86547822 -17900.50228214  41179.87189102
  59662.72919779 -21931.33872719 -48499.68956041 -20602.79583432
 -23570.97318626]


In [58]:
### Fit on a normalized model
Ridge_m1 = Ridge(alpha, fit_intercept=True, normalize=True)  ## with normalization but no standardization
Ridge_m1.fit(X_train,y_train)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

In [59]:
Ridge_m1.coef_[1:10]

array([  1.39871555e+03,  -2.83523730e+01,  -8.75308132e-01,
         0.00000000e+00,   3.49514609e+03,   3.00555304e+02,
        -2.24399066e+03,   0.00000000e+00,   2.82640392e+02])

In [60]:
Ridge_m1.intercept_

336755.64199602051

In [62]:
## check prediction
pred1=Ridge_m1.predict(X_test[1:10])
print(pred1) ### it seems like that the predictions are on the similar order

[ 42391.77507287 -30893.83112345 -14539.93962092  38126.11435998
  57760.40791581 -21258.83357354 -44444.11176297 -13352.54835861
 -24723.68171482]


In [63]:
### Fit on a standardized input but no normalization
#standardizing the input X/y
scal_y_in=scale(y_train)
print("mean=",np.mean(y_train))
print("standard dev=",np.std(y_train))
scalerX = StandardScaler(copy=True, with_mean=True, with_std=True)
scalerX.fit(X_train)
scal_X_in=scalerX.transform(X_train)
Ridge_m2 = Ridge(alpha, fit_intercept=True, normalize=False)  ## no normalization
Ridge_m2.fit(scal_X_in,scal_y_in)

mean= -2711.754932
standard dev= 45072.6612006


Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [64]:
Ridge_m2.coef_[1:10]

array([ 0.05200768, -0.02348345, -0.0759738 ,  0.        ,  0.03868858,
        0.0188621 , -0.02153909,  0.        ,  0.01461087])

In [66]:
Ridge_m2.intercept_

1.4312776820186974e-16

In [68]:
## check prediction
pred2=Ridge_m2.predict(scalerX.transform(X_test[1:10]))
print("before reverse standardization:",pred2)
pred2_rev = pred2*45072.66 -2711.754932
print("after reverse standardization:",pred2_rev) ### It is meaningful to reverse the predictions if normalize is turned off but standardization on our own

before reverse standardization: [ 1.11226949 -0.83789023 -0.33795972  0.97418067  1.38409232 -0.42635663
 -1.01690876 -0.39667402 -0.46422794]
after reverse standardization: [ 47421.18947279 -40477.6964662  -17944.49826388  41197.1590582
  59672.96746417 -21928.78223903 -48546.5378334  -20590.9081383
 -23635.74282921]


In [70]:
### Fit on a standardized input and with normalization
#standardizing the input X/y
Ridge_m3 = Ridge(alpha, fit_intercept=True, normalize=True)  ## with normalization
Ridge_m3.fit(scal_X_in,scal_y_in)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

In [71]:
Ridge_m3.coef_[1:10]   ## can see that some part of the coefficients are changed, normalization taking effect

array([ 0.01262311, -0.02087983, -0.06079894,  0.        ,  0.00643438,
        0.00913098, -0.01366541,  0.        ,  0.01073285])

In [72]:
Ridge_m3.intercept_

5.1896686919286109e-16

In [74]:
## check prediction
pred3=Ridge_m3.predict(scalerX.transform(X_test[1:10]))
print("before reverse standardization:",pred3)
pred3_rev = pred3*45072.66 -2711.754932
print("after reverse standardization:",pred3_rev) ### if normalize is turned on, with standardization, seems normalization is the ultimate effect for the model, but standardization + normalization is faily simiilar to use normalization only

before reverse standardization: [ 1.00068487 -0.62525876 -0.26242481  0.90604522  1.34165947 -0.41149287
 -0.92589068 -0.23608088 -0.48836537]
after reverse standardization: [ 42391.77387148 -30893.83037278 -14539.93930586  38126.11327221
  57760.40630505 -21258.83307951 -44444.11065137 -13352.54807517
 -24723.6811285 ]


In [77]:
## check the MSE score for all 4 models and see which one is the best

MSE_test0 = metrics.mean_squared_error(y_test[1:10], pred0)
print("no normalization or standardization:", MSE_test0)
MSE_test1 = metrics.mean_squared_error(y_test[1:10], pred1)
print("with normalization but no standardization:", MSE_test1)
MSE_test2 = metrics.mean_squared_error(y_test[1:10], pred2_rev)
print("with only standardization but no normalization:", MSE_test2)
MSE_test3 = metrics.mean_squared_error(y_test[1:10], pred3_rev)
print("with standardization and normalization:", MSE_test3)  ### winning final

no normalization or standardization: 422529975.239
with normalization but no standardization: 390486928.717
with only standardization but no normalization: 423195508.538
with standardization and normalization: 390486919.593


In [85]:
## Take a look at all neighborhoods
X_2house['Neighborhood'].value_counts()

15.0    679237
20.0    352699
5.0     332876
24.0    299461
19.0    282728
8.0     271383
7.0     252651
17.0    220702
23.0    213971
22.0    209317
3.0     158216
14.0    139413
6.0     115290
18.0    113092
11.0    110876
26.0     87023
25.0     81775
21.0     68804
4.0      60646
2.0      48362
16.0     47798
13.0     42578
0.0      40040
27.0     33065
1.0      17422
9.0      10709
10.0       710
12.0       141
Name: Neighborhood, dtype: int64

### Start training all models for stratified data

In [128]:
# Start stratifying dataset to 27 groups 
import pickle
from sklearn.externals import joblib
non_combine = [1,2,3,4,5,6,7,8,9,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]
combine =[10,12]
#some fix cv parameters
alpha_ary=np.array([0.01, 0.05 ,0.1, 0.5, 5, 10, 15, 20, 25, 30, 50])
alpha_ratio =np.array([0.5, 0.4, 0.3])

# preserved testing data
X_test_out = pd.DataFrame(columns = X_2house.columns)
y_test_out = np.array([])
#start stratifying by number
for neighbor in non_combine:
    X_codeN, y_codeN, X_codebol = select_Neigh_df(X_2house, y_2house, neighbor)
                                                  
    # do a train-test split 0.2 percent
    X_cN_train, X_cN_test, y_cN_train, y_cN_test = train_test_split(X_codeN,y_codeN,test_size=0.2,random_state=22)
    # reserve the 20% testing data , save out
    X_test_out = X_test_out.append(X_cN_test)   
    y_test_out = np.append(y_test_out, y_cN_test)                                             
    # start by search CV model selection
    df, coef_df, X_mean, X_std, y_mean, y_std = model_fit(X_cN_train, y_cN_train, alpha_ary, alpha_ratio)
    # get the best model by minimize MSE:
    best= df.loc[df['MSE_test']==df['MSE_test'].min()]
    if len(best)>1: best = best.iloc[0]  #exclude all other rows
    #start saving results
    
    if (best['model type']=='LR'):f_model = LinearRegression(fit_intercept = True, normalize=True)
    if (best['model type']=='Lasso'): f_model = Lasso(alpha=best['alpha'], fit_intercept = True, normalize=True, copy_X=True, max_iter=1000,random_state=22)
    if (best['model type']=='Ridge'): f_model = Ridge(alpha=best['alpha'], fit_intercept = True, normalize=True, copy_X=True, max_iter=1000,random_state=22)
    if (best['model type']=='ElasticNet'): f_model = ElasticNet(alpha=best['alpha'], l1_ratio=best['alpha_ratio'], fit_intercept = True, normalize=True, copy_X=True, max_iter=1000,random_state=22)    
    
    #abandon cv df:
    del df
    
    f_model.fit(X_cN_train,y_cN_train)
    # preserve model
    mod_name = 'save_mod'+str(neighbor)+'.pkl'
    joblib.dump(f_model,mod_name)
    
    # save the y-scaler info for restore 
    yscaler = np.array([y_mean, y_std])
    scal_name = 'save_scal'+str(neighbor)+'.np'
    np.save(scal_name, yscaler)

mean of prescale y: 4055.55555556
standard deviation of prescale y: 42804.7274417
cv of Lasso..... 1
cv of Ridge..... 1
cv of LR..... 1
cv of Lasso..... 2
cv of Ridge..... 2
cv of LR..... 2
cv of Lasso..... 3
cv of Ridge..... 3
cv of LR..... 3
cv of Lasso..... 1
cv of Ridge..... 1
cv of LR..... 1
cv of Lasso..... 2
cv of Ridge..... 2
cv of LR..... 2
cv of Lasso..... 3
cv of Ridge..... 3
cv of LR..... 3
cv of Lasso..... 1
cv of Ridge..... 1
cv of LR..... 1
cv of Lasso..... 2
cv of Ridge..... 2
cv of LR..... 2
cv of Lasso..... 3
cv of Ridge..... 3
cv of LR..... 3
cv of Lasso..... 1
cv of Ridge..... 1
cv of LR..... 1
cv of Lasso..... 2
cv of Ridge..... 2
cv of LR..... 2
cv of Lasso..... 3
cv of Ridge..... 3
cv of LR..... 3
cv of Lasso..... 1
cv of Ridge..... 1
cv of LR..... 1
cv of Lasso..... 2
cv of Ridge..... 2
cv of LR..... 2
cv of Lasso..... 3
cv of Ridge..... 3
cv of LR..... 3
cv of Lasso..... 1
cv of Ridge..... 1
cv of LR..... 1
cv of Lasso..... 2
cv of Ridge..... 2
cv of LR..... 2


In [143]:
# Deal with the last group, Neighborhood 10 and 12
X_N = X_2house.loc[(X_2house['Neighborhood']==10) & (X_2house['Neighborhood_2']==10)]
#X_N_bol=[(X_2house['Neighborhood']==3)& (X_2house['Neighborhood_2']==3)] 
X_N12=df.loc[(X_2house['Neighborhood']==12) & (X_2house['Neighborhood_2']==12)]
#X_N12_bol=[(X_2house['Neighborhood']==12)& (X_2house['Neighborhood_2']==12)] 
#X_N = X_N.append(X_N12)
#X_N_bol = np.append
#y_N = y[X_N_bol]

In [142]:
X_N[['Neighborhood','Neighborhood_2']]

,Neighborhood,Neighborhood_2
Order,,
4064819,10.0,10.0


In [149]:
X_orig = pd.read_excel('Data/Raw/AmesHousing.xls', 'Sheet1')

In [156]:
X_N = X_orig.loc[(X_orig['Neighborhood']=='GrnHill')]

In [157]:
X_N[['Neighborhood','MS SubClass','MS Zoning','Bldg Type','House Style']]

,Neighborhood,MS SubClass,MS Zoning,Bldg Type,House Style
2256,GrnHill,120,RM,TwnhsE,1Story
2892,GrnHill,120,RM,TwnhsE,1Story


In [155]:
X_orig['Neighborhood'].value_counts()

NAmes      443
CollgCr    267
OldTown    239
Edwards    194
Somerst    182
NridgHt    166
Gilbert    165
Sawyer     151
NWAmes     131
SawyerW    125
Mitchel    114
BrkSide    108
Crawfor    103
IDOTRR      93
Timber      72
NoRidge     71
StoneBr     51
SWISU       48
ClearCr     44
MeadowV     37
BrDale      30
Blmngtn     28
Veenker     24
NPkVill     23
Blueste     10
Greens       8
GrnHill      2
Landmrk      1
Name: Neighborhood, dtype: int64

In [118]:
best

test#                   15
model type              LR
alpha                  NAN
alpha_ratio            NAN
MSE_test       3.82145e-29
MSE_train      4.03041e-29
intercept      -0.00849877
cv#                      3
Name: 14, dtype: object

In [119]:
df

,test#,model type,alpha,alpha_ratio,MSE_test,MSE_train,intercept,cv#
0,1,Lasso,0.01,NAN,9.800150e-01,9.792063e-01,-0.014069,1
1,2,Ridge,0.01,NAN,1.202074e-03,1.774006e-05,-0.014069,1
2,3,LR,NAN,NAN,5.348039e-03,4.808523e-30,-0.014069,1
3,4,ElasticNet,0.01,0.5,6.451633e-01,6.297853e-01,-0.014069,1
4,5,ElasticNet,0.01,0.4,6.451633e-01,6.297853e-01,-0.014069,1
5,6,ElasticNet,0.01,0.3,6.451633e-01,6.297853e-01,-0.014069,1
6,7,Lasso,0.01,NAN,1.089132e+00,9.561980e-01,0.022568,2
7,8,Ridge,0.01,NAN,2.439163e-03,9.320175e-06,0.022568,2
8,9,LR,NAN,NAN,2.394409e-03,3.449794e-30,0.022568,2
9,10,ElasticNet,0.01,0.5,7.127419e-01,6.420845e-01,0.022568,2


In [26]:
titanic = pd.read_csv("https://raw.githubusercontent.com/amueller/scipy-2017-sklearn/master/notebooks/datasets/titanic3.csv")

In [27]:
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [30]:
tit_dum=pd.get_dummies(titanic)

In [31]:
print(tit_dum)

      pclass  survived      age  sibsp  parch      fare   body  \
0          1         1  29.0000      0      0  211.3375    NaN   
1          1         1   0.9167      1      2  151.5500    NaN   
2          1         0   2.0000      1      2  151.5500    NaN   
3          1         0  30.0000      1      2  151.5500  135.0   
4          1         0  25.0000      1      2  151.5500    NaN   
5          1         1  48.0000      0      0   26.5500    NaN   
6          1         1  63.0000      1      0   77.9583    NaN   
7          1         0  39.0000      0      0    0.0000    NaN   
8          1         1  53.0000      2      0   51.4792    NaN   
9          1         0  71.0000      0      0   49.5042   22.0   
10         1         0  47.0000      1      0  227.5250  124.0   
11         1         1  18.0000      1      0  227.5250    NaN   
12         1         1  24.0000      0      0   69.3000    NaN   
13         1         1  26.0000      0      0   78.8500    NaN   
14        